## Intro Aprendizaje por Refuerzo

Considera el escenario de enseñarle nuevos trucos a un perro. El perro no entiende nuestro lenguaje, así que no podemos decirle qué hacer. En cambio, seguimos una estrategia diferente. Emulamos una situación (o una señal), y el perro intenta responder de muchas maneras diferentes. Si la respuesta del perro es la deseada, lo recompensamos con golosinas. Ahora, adivina qué, la próxima vez que el perro se enfrenta a la misma situación, ejecuta una acción similar con aún más entusiasmo esperando más comida. Esto es como aprender "qué hacer" a partir de experiencias positivas. De manera similar, los perros tienden a aprender qué no hacer cuando se enfrentan a experiencias negativas.


## Entendiendo cómo funciona el Aprendizaje por Refuerzo

En un sentido más amplio, así es como funciona el Aprendizaje por Refuerzo:

* Tu perro es un "agente" que está expuesto al entorno. El entorno podría ser tu casa, contigo.
* Las situaciones que encuentran son análogas a un estado. Un ejemplo de un estado podría ser tu perro de pie y tú usando una palabra específica con cierto tono en tu sala de estar.
* Nuestros agentes reaccionan realizando una acción para pasar de un "estado" a otro "estado"; por ejemplo, tu perro pasa de estar de pie a sentarse.
* Después de la transición, pueden recibir una recompensa o una penalización a cambio. ¡Les das una golosina! O un "No" como penalización.
* La política es la estrategia de elegir una acción dada un estado en espera de mejores resultados.


El Aprendizaje por Refuerzo se encuentra entre el espectro del Aprendizaje Supervisado y el No Supervisado, y hay algunas cosas importantes que tener en cuenta:

#### No siempre es beneficioso ser codicioso

**Ser codicioso no siempre funciona:**

* Hay cosas que son fáciles de hacer para obtener una gratificación instantánea, y hay cosas que proporcionan recompensas a largo plazo. El objetivo no es ser codicioso buscando las recompensas inmediatas, sino optimizar las recompensas máximas durante todo el entrenamiento.

#### La secuencia importa en el Aprendizaje por Refuerzo

**La secuencia importa en el Aprendizaje por Refuerzo:**

* La recompensa del agente no solo depende del estado actual, sino de toda la historia de estados. A diferencia del aprendizaje supervisado y no supervisado, el tiempo es importante aquí.


## El proceso

<img src="./img/Reinforcement-Learning-Animation.gif" alt="drawing" width="650"/>

En cierto sentido, el Aprendizaje por Refuerzo es la ciencia de tomar decisiones óptimas utilizando experiencias. Desglosándolo, el proceso de Aprendizaje por Refuerzo involucra estos pasos simples:

1. Observación del entorno.
2. Decidir cómo actuar utilizando alguna estrategia.
3. Actuar en consecuencia.
4. Recibir una recompensa o penalización.
5. Aprender de las experiencias y refinar nuestra estrategia.
6. Iterar hasta encontrar una estrategia óptima.

Ahora vamos a entender el Aprendizaje por Refuerzo desarrollando un agente para aprender a jugar un juego automáticamente por sí mismo.

## Comenzando con Gym

Gym es un conjunto de herramientas para desarrollar y comparar algoritmos de aprendizaje por refuerzo. No hace suposiciones sobre la estructura de tu agente y es compatible con cualquier biblioteca de cálculo numérico, como TensorFlow o Theano.

La biblioteca Gym es una colección de problemas de prueba, o entornos, que puedes usar para desarrollar tus algoritmos de aprendizaje por refuerzo. Estos entornos tienen una interfaz compartida, lo que te permite escribir algoritmos generales.


In [1]:
# pip install gym
# pip install pygame

## Entornos

Aquí tienes un ejemplo mínimo para comenzar a ejecutar algo. Esto ejecutará una instancia del entorno CartPole-v0 durante 1000 pasos de tiempo, representando el entorno en cada paso.

 Consiste en equilibrar un poste (péndulo) montado en la parte superior de un carrito móvil:
 * El **objetivo** es mantener el poste en posición vertical mientras el carrito se mueve hacia adelante y hacia atrás en una pista. 
 * El **agente** (o jugador) tiene dos acciones disponibles en cada paso de tiempo: empujar el carrito hacia la izquierda o hacia la derecha. 
 * El **desafío** radica en tomar decisiones adecuadas para evitar que el poste caiga mientras se mueve el carrito. 
 
 Este problema es un ejemplo comúnmente utilizado para probar algoritmos de aprendizaje por refuerzo debido a su simplicidad y naturaleza desafiante.

https://gymnasium.farama.org/api/env/

In [1]:
import gym
import warnings
warnings.filterwarnings("ignore")

# import time

env = gym.make('MountainCar-v0', render_mode="human")
env.reset()
for i in range(300):
    env.render()
    # time.sleep(0.1)
    env.step(env.action_space.sample()) # take a random action
env.close()

Normalmente, terminaremos la simulación antes de que el carrito-péndulo tenga permitido salir de la pantalla. Más sobre eso después. Por ahora, por favor ignora la advertencia sobre llamar a `step()` aunque este entorno ya haya devuelto `done = True`.

Si deseas ver otros entornos en acción, intenta reemplazar `CartPole-v0` arriba con algo como `MountainCar-v0`, `MsPacman-v0` (requiere la dependencia de Atari), o `Hopper-v1` (requiere las dependencias de MuJoCo). Todos los entornos descienden de la clase base `Env`.

Ten en cuenta que si te faltan dependencias, deberías recibir un mensaje de error útil que te diga qué te falta. (Avísanos si alguna dependencia te causa problemas sin una instrucción clara para solucionarlo). Instalar una dependencia faltante generalmente es bastante simple. También necesitarás una licencia de MuJoCo para `Hopper-v1`.


## Observations

## Observaciones

Si queremos hacer algo mejor que tomar acciones aleatorias en cada paso, probablemente sería bueno saber realmente qué están haciendo nuestras acciones en el entorno.

La función `step` del entorno devuelve exactamente lo que necesitamos. De hecho, `step` devuelve cuatro valores. Estos son:

* `observation` (object): un objeto específico del entorno que representa tu observación del entorno. Por ejemplo, datos de píxeles de una cámara, ángulos de articulación y velocidades de articulación de un robot, o el estado del tablero en un juego de mesa.
* `reward` (float): cantidad de recompensa lograda por la acción anterior. La escala varía entre entornos, pero el objetivo es siempre aumentar tu recompensa total.
* `done` (bool): si es hora de restablecer el entorno nuevamente. La mayoría (pero no todos) de las tareas están divididas en episodios bien definidos, y `done` siendo True indica que el episodio ha terminado. (Por ejemplo, tal vez el poste se inclinó demasiado, o perdiste tu última vida).
* `truncated` (bool): True si el episodio se trunca debido a un límite de tiempo o una razón que no está definida.
* `info` (dict): información de diagnóstico útil para la depuración. A veces puede ser útil para el aprendizaje (por ejemplo, podría contener las probabilidades crudas detrás del último cambio de estado del entorno). Sin embargo, no se permite usar esto para el aprendizaje en las evaluaciones oficiales de tu agente.

Esto es simplemente una implementación del clásico "bucle agente-entorno". En cada paso de tiempo, el agente elige una acción, y el entorno devuelve una observación y una recompensa.


In [6]:
env = gym.make('MountainCar-v0')

In [7]:
env.action_space

Discrete(3)

## Espacio de Acciones

Hay 3 acciones discretas determinísticas:

| Num | Observación          | Valor | Unidad       |
|-----|----------------------|-------|--------------|
| 0   | Acelerar a la izquierda | Inf   | posición (m) |
| 1   | No acelerar              | Inf   | posición (m) |
| 2   | Acelerar a la derecha    | Inf   | posición (m) |


In [8]:
# env = gym.make('MountainCar-v0', render_mode='human')
print(env.observation_space.shape)

(2,)


## Espacio de Observación

La observación es un ndarray con forma (2,), donde los elementos corresponden a lo siguiente:

| Num | Observación                    | Mínimo | Máximo | Unidad        |
|-----|--------------------------------|--------|--------|---------------|
| 0   | posición del coche a lo largo del eje x | -Inf   | Inf    | posición (m) |
| 1   | velocidad del coche             | -Inf   | Inf    | velocidad (m) |


In [6]:
import numpy as np
np.random.choice(np.array([1,2,3]))

2

In [2]:
# import gym
env = gym.make('MountainCar-v0', render_mode='human')
for i_episode in range(3):
    # Reset del entorno
    print("Intento", i_episode)
    observation = env.reset()
    for t in range(100):
        if t % 10 == 0:
            print("Accion", t)
            print("Observación", observation)
        # Visualizar ejercicio
        env.render()
        # Print del estado
        # print(observation)
        # Guarda en la variable action una de las acciones posibles elegida al azar
        action = env.action_space.sample()
        # Ejecuta esa acción, lo que nos devuelve, un nuevo estado, una recompensa, booleana que nos indica que ha alcanzado su objetivo e info debug
        observation, reward, done, truncated, info = env.step(action)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break
env.close()

Intento 0
Accion 0
Observación (array([-0.43425947,  0.        ], dtype=float32), {})
Accion 10
Observación [-0.45745373 -0.00194966]
Accion 20
Observación [-0.49368823 -0.00709359]
Accion 30
Observación [-0.5528266 -0.0041555]
Accion 40
Observación [-0.5682742  -0.00143375]
Accion 50
Observación [-0.56198674  0.00190312]
Accion 60
Observación [-0.5233326   0.00578599]
Accion 70
Observación [-0.47611013  0.00302374]
Accion 80
Observación [-0.4635225  -0.00127068]
Accion 90
Observación [-0.477988  -0.0022708]
Intento 1
Accion 0
Observación (array([-0.4816161,  0.       ], dtype=float32), {})
Accion 10
Observación [-0.48506343 -0.00115321]
Accion 20
Observación [-0.51441854 -0.00418148]
Accion 30
Observación [-0.53665835 -0.00199559]
Accion 40
Observación [-0.5412562   0.00141854]
Accion 50
Observación [-0.51583534  0.00409613]
Accion 60
Observación [-0.4704284   0.00404052]
Accion 70
Observación [-0.4785057 -0.0019512]
Accion 80
Observación [-0.4912765  -0.00195884]
Accion 90
Observació